# Разиение выборки на подвыборки

##### Автор: Виктор Китов ([DeepMachineLearning.ru](https://deepmachinelearning.ru))

##### Лицензия: BSD-3-Clause

В машинном обучении исходную выборки объектов нужно разделять на обучающую и валидационную подвыборку.
- На обучающей выборке проходит настройка **параметров** модели
- При настроенных параметрах модель прогнозирует данные в **валидационной** выборке. По качеству этих прогнозов подбираются структурные настройки модели, определяемые **гиперпараметрами**.

На больших данных разбиение может происходить однократно, поскольку каждая подвыборка тоже будет получаться большой, а оценка по ней - стабильной (по закону больших чисел). В случае же небольших выборок, подобное разбиение рекомендуется производить **много раз** и прогнозирующее качество модели **усреднять по всевозможным разбиениям**. Это позволяет получить более стабильную и надежную оценку качества модели, уменьшая зависимость от конкретного случайного разбиения данных. Этот процесс называется кросс-валидацией.

Далее будут рассмотрены основные способы разбиения исходной выборки объектов на подвыборки, доступные в библиотеке `scikit-learn`.

In [1]:
import numpy as np
from sklearn.model_selection import (
    KFold, 
    RepeatedKFold, 
    LeaveOneOut, 
    ShuffleSplit, 
    StratifiedKFold
)

# Создадим просты демо-данные

In [2]:
# Создание простых данных для демонстрации
X = np.array([[i, i+1] for i in range(10, 101, 10)])
Y = np.array([1, -1] * 5)  # Чередующиеся метки: 1, -1, 1, -1, ...
print("X shape:", X.shape)
print("Y shape:", Y.shape)
print("\nX данные:")
print(X)
print("\nY метки:")
print(Y)

X shape: (10, 2)
Y shape: (10,)

X данные:
[[ 10  11]
 [ 20  21]
 [ 30  31]
 [ 40  41]
 [ 50  51]
 [ 60  61]
 [ 70  71]
 [ 80  81]
 [ 90  91]
 [100 101]]

Y метки:
[ 1 -1  1 -1  1 -1  1 -1  1 -1]


# KFold - K-кратная кросс-валидация

**KFold** делит данные на K равных частей (фолдов). В каждой итерации одна часть используется для тестирования, остальные K-1 - для обучения. Процесс повторяется K раз, каждый раз с разном тестовой частью.

In [3]:
print("=" * 50)
print("1. KFold кросс-валидация (K=5)")
print("=" * 50)

kf = KFold(n_splits=5, shuffle=True, random_state=42)
for fold, (train_idx, test_idx) in enumerate(kf.split(X), 1):
    print(f"\nФолд {fold}:")
    print(f"  Обучение: индексы {train_idx}")
    print(f"  Тестирование: индексы {test_idx}")
    print(f"  Тест выборка X: {X[test_idx].tolist()}")
    print(f"  Тест выборка Y: {Y[test_idx].tolist()}")

1. KFold кросс-валидация (K=5)

Фолд 1:
  Обучение: индексы [0 2 3 4 5 6 7 9]
  Тестирование: индексы [1 8]
  Тест выборка X: [[20, 21], [90, 91]]
  Тест выборка Y: [-1, 1]

Фолд 2:
  Обучение: индексы [1 2 3 4 6 7 8 9]
  Тестирование: индексы [0 5]
  Тест выборка X: [[10, 11], [60, 61]]
  Тест выборка Y: [1, -1]

Фолд 3:
  Обучение: индексы [0 1 3 4 5 6 8 9]
  Тестирование: индексы [2 7]
  Тест выборка X: [[30, 31], [80, 81]]
  Тест выборка Y: [1, -1]

Фолд 4:
  Обучение: индексы [0 1 2 3 5 6 7 8]
  Тестирование: индексы [4 9]
  Тест выборка X: [[50, 51], [100, 101]]
  Тест выборка Y: [1, -1]

Фолд 5:
  Обучение: индексы [0 1 2 4 5 7 8 9]
  Тестирование: индексы [3 6]
  Тест выборка X: [[40, 41], [70, 71]]
  Тест выборка Y: [-1, 1]


# RepeatedKFold - повторная K-кратная кросс-валидация

**RepeatedKFold** повторяет KFold несколько раз с различными случайными разбиениями. Это полезно для получения более стабильной оценки модели, уменьшая зависимость от конкретного разбиения.

In [4]:
print("\n" + "=" * 50)
print("2. RepeatedKFold кросс-валидация")
print("=" * 50)

rkf = RepeatedKFold(n_splits=3, n_repeats=2, random_state=42)
for repeat, (train_idx, test_idx) in enumerate(rkf.split(X), 1):
    print(f"\nПовтор {repeat}:")
    print(f"  Обучение: индексы {train_idx}")
    print(f"  Тестирование: индексы {test_idx}")


2. RepeatedKFold кросс-валидация

Повтор 1:
  Обучение: индексы [2 3 4 6 7 9]
  Тестирование: индексы [0 1 5 8]

Повтор 2:
  Обучение: индексы [0 1 3 4 5 6 8]
  Тестирование: индексы [2 7 9]

Повтор 3:
  Обучение: индексы [0 1 2 5 7 8 9]
  Тестирование: индексы [3 4 6]

Повтор 4:
  Обучение: индексы [2 3 4 6 7 9]
  Тестирование: индексы [0 1 5 8]

Повтор 5:
  Обучение: индексы [0 1 2 5 6 8 9]
  Тестирование: индексы [3 4 7]

Повтор 6:
  Обучение: индексы [0 1 3 4 5 7 8]
  Тестирование: индексы [2 6 9]


# LeaveOneOut - кросс-валидация с исключением одного элемента

**LeaveOneOut** - это частный случай KFold, где K равно количеству объектов. На каждой итерации один объект используется для тестирования, а все остальные - для обучения. Это очень затратный метод, но полезный для маленьких датасетов.

In [6]:
print("\n" + "=" * 50)
print("3. LeaveOneOut кросс-валидация")
print("=" * 50)

loo = LeaveOneOut()
# Покажем только первые 5 итераций для краткости
for i, (train_idx, test_idx) in enumerate(loo.split(X)):
    if i < 5:  # Показать только первые 5
        print(f"\nИтерация {i+1}:")
        print(f"  Тестовый индекс: {test_idx[0]}")
        print(f"  X тест: {X[test_idx].tolist()}, Y тест: {Y[test_idx].tolist()}")
        print(f"  Размер обучения: {len(train_idx)}, размер теста: {len(test_idx)}")
    else:
        continue

print(f"\nВсего итераций: {X.shape[0]}")


3. LeaveOneOut кросс-валидация

Итерация 1:
  Тестовый индекс: 0
  X тест: [[10, 11]], Y тест: [1]
  Размер обучения: 9, размер теста: 1

Итерация 2:
  Тестовый индекс: 1
  X тест: [[20, 21]], Y тест: [-1]
  Размер обучения: 9, размер теста: 1

Итерация 3:
  Тестовый индекс: 2
  X тест: [[30, 31]], Y тест: [1]
  Размер обучения: 9, размер теста: 1

Итерация 4:
  Тестовый индекс: 3
  X тест: [[40, 41]], Y тест: [-1]
  Размер обучения: 9, размер теста: 1

Итерация 5:
  Тестовый индекс: 4
  X тест: [[50, 51]], Y тест: [1]
  Размер обучения: 9, размер теста: 1

Всего итераций: 10


# ShuffleSplit - случайное разбиение

**ShuffleSplit** создает случайные разбиения данных. В отличие от KFold, здесь нет гарантии, что все данные будут использованы для тестирования. Можно задать количество разбиений и размер тестовой выборки.

In [7]:
print("\n" + "=" * 50)
print("4. ShuffleSplit (Randomized Split)")
print("=" * 50)

ss = ShuffleSplit(n_splits=3, test_size=0.3, random_state=42)
for split, (train_idx, test_idx) in enumerate(ss.split(X), 1):
    print(f"\nРазбиение {split}:")
    print(f"  Обучение: индексы {train_idx}")
    print(f"  Тестирование: индексы {test_idx}")
    print(f"  Размер теста: {len(test_idx)}/{len(X)} ({len(test_idx)/len(X)*100:.0f}%)")


4. ShuffleSplit (Randomized Split)

Разбиение 1:
  Обучение: индексы [0 7 2 9 4 3 6]
  Тестирование: индексы [8 1 5]
  Размер теста: 3/10 (30%)

Разбиение 2:
  Обучение: индексы [5 3 4 7 9 6 2]
  Тестирование: индексы [0 1 8]
  Размер теста: 3/10 (30%)

Разбиение 3:
  Обучение: индексы [6 8 5 3 7 1 4]
  Тестирование: индексы [9 2 0]
  Размер теста: 3/10 (30%)


# StratifiedKFold - K-кратная кросс-валидация с сохранением распределений

**StratifiedKFold** сохраняет распределение классов (или других дискретных величин) в каждом блоке кросс-валидации таким же, как в исходном наборе данных. Это особенно важно при работе с несбалансированными данными.

In [11]:
print("\n" + "=" * 50)
print("5. StratifiedKFold кросс-валидация")
print("=" * 50)

# Посмотрим распределение классов
unique, counts = np.unique(Y, return_counts=True)
class_proportions = counts/len(Y)

print(f"Исходное распределение классов:")
print(f"  Класс -1: {class_proportions[0]*100:.0f}%")
print(f"  Класс 1: {class_proportions[1]*100:.0f}%")

skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
for fold, (train_idx, test_idx) in enumerate(skf.split(X, Y), 1):
    print(f"\nБлок {fold}:")
    print(f"  Обучение: индексы {train_idx}")
    print(f"  Тестирование: индексы {test_idx}")
    
    # Проверим распределение классов в блоках
    train_unique, train_counts = np.unique(Y[train_idx], return_counts=True)
    test_unique, test_counts = np.unique(Y[test_idx], return_counts=True)
    
    train_proportions = train_counts/len(train_idx)
    test_proportions = test_counts/len(test_idx)
    
    print(f"  Обучение распределение:")
    print(f"    Класс -1: {train_proportions[0]*100:.0f}%")
    print(f"    Класс 1: {train_proportions[1]*100:.0f}%")
    
    print(f"  Тестирование распределение:")
    print(f"    Класс -1: {test_proportions[0]*100:.0f}%")
    print(f"    Класс 1: {test_proportions[1]*100:.0f}%")


5. StratifiedKFold кросс-валидация
Исходное распределение классов:
  Класс -1: 50%
  Класс 1: 50%

Блок 1:
  Обучение: индексы [1 2 4 5 8 9]
  Тестирование: индексы [0 3 6 7]
  Обучение распределение:
    Класс -1: 50%
    Класс 1: 50%
  Тестирование распределение:
    Класс -1: 50%
    Класс 1: 50%

Блок 2:
  Обучение: индексы [0 1 2 3 6 7 9]
  Тестирование: индексы [4 5 8]
  Обучение распределение:
    Класс -1: 57%
    Класс 1: 43%
  Тестирование распределение:
    Класс -1: 33%
    Класс 1: 67%

Блок 3:
  Обучение: индексы [0 3 4 5 6 7 8]
  Тестирование: индексы [1 2 9]
  Обучение распределение:
    Класс -1: 43%
    Класс 1: 57%
  Тестирование распределение:
    Класс -1: 67%
    Класс 1: 33%


# Заключение

**KFold**: Разбивает данные на K равных частей, каждая часть становится тестовой ровно 1 раз

**RepeatedKFold**: Повторяет KFold несколько раз с разными случайными разбиениями для стабильности

**LeaveOneOut**: Каждый образец по очереди становится тестовым (K равно числу объектов)

**ShuffleSplit**: Случайные разбиения, тестовая выборка может пересекаться между итерациями

**StratifiedKFold**: Сохраняет распределение классов в каждом фолде как в исходных данных